In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [2]:
%pip install -U trl

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/226.7 kB ? eta -:--:--
   ---------------------------------------- 226.7/226.7 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/102.4 kB ? eta -:--:--
   ---------------------------------------- 102.4/102.4 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/547.8 kB ? eta -:--:--
   --------------------------------------- 547.8/547.8 kB 16.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/371.6 kB ? eta -:--:--
   ---------------------------------------- 371.6/371.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
   -- ------------------------------------- 1.8/25.9 MB 57.8 MB/s eta 0:00:01
   --- -------------------------------

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
base_model = r"C:\Users\satwi\Downloads\ChatBot\Models\llama-3-transformers-8b-chat-hf-v1"
new_model = r"C:\Users\satwi\Downloads\ChatBot\Models\llama"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)

model = model.merge_and_unload()

c:\Users\satwi\anaconda3\envs\llama\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:23<00:00,  5.95s/it]
c:\Users\satwi\anaconda3\envs\llama\lib\site-packages\torch\nn\modules\module.py:2047: UserWarning: for base_model.model.model.layers.7.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
c:\Users\satwi\an

ZeroDivisionError: integer division or modulo by zero

In [7]:
messages = [{"role": "user", "content": "I want to replace my oreder"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=120, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|im_start|>user
I want to replace my oreder<|im_end|>
<|im_start|>assistant
I appreciate your desire to replace your order with the number. We apologize for any inconvenience caused. To proceed with the replacement, could you please provide us with the details of the order you would like to replace it with? This will help us ensure a seamless replacement process and provide you with the order you want. Thank you for your patience and cooperation.assistant
I appreciate your desire to replace your order with the number. We apologize for any inconvenience caused. To proceed with the replacement, could you please provide us with the details of the order you would like to replace it with? This will


In [8]:
model.save_pretrained("llama-3-8b-chat-doctor")
tokenizer.save_pretrained("llama-3-8b-chat-doctor")

('llama-3-8b-chat-doctor/tokenizer_config.json',
 'llama-3-8b-chat-doctor/special_tokens_map.json',
 'llama-3-8b-chat-doctor/tokenizer.json')

In [9]:
model.push_to_hub("llama-3-8b-chat-doctor", use_temp_dir=False)
tokenizer.push_to_hub("llama-3-8b-chat-doctor", use_temp_dir=False)

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Alkyema/llama-3-8b-chat-doctor/commit/69d4e1e4d3086a718e0e7e8e2646500b8ac94ed7', commit_message='Upload tokenizer', commit_description='', oid='69d4e1e4d3086a718e0e7e8e2646500b8ac94ed7', pr_url=None, pr_revision=None, pr_num=None)

/kaggle/working


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'llama.cpp'...
remote: Enumerating objects: 966, done.
remote: Counting objects: 100% (966/966), done.
remote: Compressing objects: 100% (735/735), done.
remote: Total 966 (delta 240), reused 602 (delta 189), pack-reused 0
Receiving objects: 100% (966/966), 18.18 MiB | 22.59 MiB/s, done.
Resolving deltas: 100% (240/240), done.
/kaggle/working/llama.cpp


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
